## Data Processing - Tensorflow

Using the Random Sample data

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [3]:
data = pd.read_csv('/Users/ananyajain/Desktop/CSC413/CSC413-Final-Project/archive/sample_labels.csv')

In [4]:
Labels_before=data.loc[:,'Finding Labels']

In [5]:
Labels_after = []
for i in range(len(Labels_before)):
   split_labels=Labels_before[i].split("|")
   if len(split_labels)==1:
      Labels_after.append(split_labels)
   else:
      lab=[]
      for j in range(len(split_labels)):
         lab.append(split_labels[j])
      Labels_after.append(lab)

In [6]:
# Connecting actual images to the labels from CSV file.
image_size=224
image_file_path='/Users/ananyajain/Desktop/CSC413/CSC413-Final-Project/archive/sample/images/'
Labels=[]
scans=[]
for i in tqdm(range(len(data["Image Index"]))):
   image=cv2.imread(image_file_path+data["Image Index"][i])
   if len(image.shape)>2: 
      resize_image=cv2.resize(image,(image_size,image_size)) 
      scans.append(resize_image[:,:,:4]) 
   else :
      image=np.dstack([image] * 3)
      resize_image=cv2.resize(image,(image_size,image_size))
      scans.append(resize_image)
print(set([x.shape for x in scans]))

  0%|          | 0/5606 [00:00<?, ?it/s]

100%|██████████| 5606/5606 [00:51<00:00, 108.06it/s]


{(224, 224, 3)}


In [7]:
classes={
0:"Hernia",
1:"Pneumonia",
2:"Fibrosis",
3:"Edema",
4:"Emphysema",
5:"Cardiomegaly",
6:"Pleural_Thickening",
7:"Consolidation",
8:"Pneumothorax",
9:"Mass",
10:"Nodule",
11:"Atelectasis",
12:"Effusion",
13:"Infiltration",
14:"No Finding",
}
def get_class(code):
   return classes[code]
def get_code(labels):
   for key,value  in classes.items():
      if value ==labels:
         return key

In [8]:
for i in tqdm(range(len(Labels_after))):
    Labels_after[i]=[get_code(x) for x in Labels_after[i]]
print(Labels_after)

100%|██████████| 5606/5606 [00:00<00:00, 971743.12it/s]

[[4, 13, 6, 8], [5, 4], [14], [11], [5, 3, 12], [7, 9], [14], [14], [12], [14], [7, 12, 13, 10], [9], [14], [14], [12], [12], [12], [14], [9, 8], [5, 7], [7], [5, 7], [5, 7], [8], [14], [12], [7, 6], [13, 10], [14], [11], [11], [2], [13], [13], [14], [14], [14], [14], [2], [14], [7], [4], [14], [14], [13], [7, 9], [14], [11, 13, 9, 6], [13], [14], [14], [13], [14], [14], [14], [14], [14], [14], [14], [13, 10], [14], [4], [14], [13], [14], [14], [8], [4, 9], [14], [14], [14], [14], [14], [5, 4], [14], [14], [14], [14], [14], [13], [8], [4, 8], [4, 13], [14], [14], [14], [14], [13], [7, 12, 13], [7, 2], [14], [2, 6], [14], [14], [14], [12, 13], [2], [14], [14], [14], [10], [10], [9, 6], [13, 6], [12, 13], [14], [14], [14], [9], [14], [14], [14], [14], [14], [9], [12], [14], [14], [11], [14], [6], [14], [14], [14], [4], [14], [14], [14], [2], [14], [12], [14], [7, 13, 8], [14], [14], [14], [14], [14], [14], [5, 12], [14], [14], [6], [14], [12], [11], [13], [8], [14], [11, 7, 12], [14], [1

In [9]:
mlp=MultiLabelBinarizer()
Labels=mlp.fit_transform(Labels_after)
scans=np.array(scans)
Labels=np.array(Labels)

In [10]:
print(scans.shape, Labels.shape)

(5606, 224, 224, 3) (5606, 15)


In [11]:
X_train, y_train, X_test, y_test = iterative_train_test_split(scans, Labels, test_size = 0.2 )
X_val, y_val, X_test, y_test = iterative_train_test_split(X_test, y_test, test_size = 0.7)
print("X_train shape",X_train.shape)
print("y_train shape",y_train.shape)
print("X_val shape",X_val.shape)
print("y_val shape",y_val.shape)
print("X_test shape",X_test.shape)
print("y_test shape",y_test.shape)

NameError: name 'iterative_train_test_split' is not defined

In [ ]:
transform = ImageDataGenerator(
rotation_range=20,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.1,
zoom_range=0.1,
horizontal_flip=False,
vertical_flip=True,
fill_mode='nearest')

In [ ]:
batch_size=16
train_transform=transform.flow(X_train,y_train,batch_size=batch_size)
val_transform=transform.flow(X_val,y_val,batch_size=batch_size)
test_transform=transform.flow(X_test,y_test,batch_size=batch_size)